# Vistelius text digitalization

In [1]:
from tika import parser
import numpy as np
import pandas as pd
from pandas.api.types import is_numeric_dtype, is_string_dtype

import os

# Parallellization modules
from joblib import Parallel, delayed
import multiprocessing

In [2]:
pdf_text = "../_DATA/Vistelius_scans_text_OCR.pdf"

In [12]:
from PyPDF2 import PdfFileWriter, PdfFileReader

def convert_pdf_into_pages(pdf, output_loc):
    inputpdf = PdfFileReader(open(pdf, "rb"))
    filename = pdf.split("/")[-1].split(".")[0]
    
    for i in range(inputpdf.numPages):
        output = PdfFileWriter()
        output.addPage(inputpdf.getPage(i))
        
        if i < 10:
            page_format = f"00{i}"
        elif i < 100:
            page_format = f"0{i}"
        else:
            page_format = f"{i}"
            
        with open(f"{output_loc}/{filename}_page_{page_format}.pdf", "wb") as outputStream:
            output.write(outputStream)
            
# Reference: https://stackoverflow.com/questions/490195/split-a-multi-page-pdf-file-into-multiple-pdf-files-with-python

In [13]:
convert_pdf_into_pages(pdf_text, "../_DATA/text_pages/")

In [3]:
pdf_parsed = parser.from_file(pdf_text)

2019-04-10 11:11:19,614 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.19/tika-server-1.19.jar to C:\Users\Bram\AppData\Local\Temp\tika-server.jar.
2019-04-10 11:11:27,001 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.19/tika-server-1.19.jar.md5 to C:\Users\Bram\AppData\Local\Temp\tika-server.jar.md5.
2019-04-10 11:11:27,449 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [ ]:
# To Do: Find way to split lines per sample, not based on newline

In [6]:
text_parsed = pdf_parsed['content'].strip().replace("\n\n", "\n").split("\n")

In [7]:
test_part = pdf_parsed['content'].strip().replace("\n\n", "\n").split("\n")[:10]

In [8]:
test_part[0]

'1- (44°31\'30",138 37\'30"). Granite alaskitic. K2 (V.N.Musin,1970). '

In [9]:
import re

def correct_lines(lines):
    regex = re.compile("[0-9]+[ ]?-")

    actual_lines = []

    for index, test in enumerate(lines):
        try: 
            regex.search(test).group()
            actual_lines.append(test)
        except:
            actual_lines[-1] = lines[index - 1] + lines[index]

    return actual_lines

In [14]:
for item in correct_lines(test_part):
    print(item.split("."))

['1- (44°31\'30",138 37\'30")', ' Granite alaskitic', ' K2 (V', 'N', 'Musin,1970)', ' ']
['2- (54°12\'10" ,119°24\')', ' Granite leucocratic', ' Tr', ' V', 'I', 'Zhigalova (E', 'A', 'Ivanov,1969)', ' ']
["3- (62°36',155°36')", ' Granite leucocratic coarse-grained', " K2'", ' Left-Omsukchan massif', ' P', 'M', 'Bosek (O', 'S', 'Gracheva,1948)', ' ']
["4- (61°35',146°02')", ' · Granite leucocratic', ' K1', ' Buksandzhin massif', ' A', 'Kh', 'Brovtman (A', 'F', 'Mikhaylov,1948)', ' ']
["5- (68°55',164°24')", ' Granite-porphyry micropegmatitic', ' K1• Attykveem massif', ' L', 'G', 'Semenova', ' 0th', ' :S-0', '16 (A', ' I', 'Sadovsky, 1963)', ' ']
["6- (50°42',113°42')", ' Bt granite porphyraceous medium-coarse-grained', ' J3', ' Sakhanay massif O', 'S', "Beloborod'ko (N", 'N', 'Chaban,1966)', ' ']


In [16]:
correct_lines(test_part)

['1- (44°31\'30",138 37\'30"). Granite alaskitic. K2 (V.N.Musin,1970). ',
 '2- (54°12\'10" ,119°24\'). Granite leucocratic. Tr. V.I.Zhigalova (E.A.Ivanov,1969). ',
 "3- (62°36',155°36'). Granite leucocratic coarse-grained. K2'. Left-Omsukchan massif. P.M.Bosek (O.S.Gracheva,1948). ",
 "4- (61°35',146°02'). · Granite leucocratic. K1. Buksandzhin massif. A.Kh.Brovtman (A.F.Mikhaylov,1948). ",
 "5- (68°55',164°24'). Granite-porphyry micropegmatitic. K1• Attykveem massif. L.G.Semenova. 0th. :S-0.16 (A. I.Sadovsky, 1963). ",
 "6- (50°42',113°42'). Bt granite porphyraceous medium-coarse-grained. J3. Sakhanay massif O.S.Beloborod'ko (N.N.Chaban,1966). "]

In [17]:
regex_sample = re.compile("[0-9]+-")

In [34]:
found_sample_numbers = []
error_counts = 0

for item in correct_lines(correct_lines(text_parsed)):
    try:
        match_sample = regex_sample.search(item).group()
        found_sample_numbers.append(int(match_sample.strip("-")))
    except:
        print("error", correct_lines(correct_lines(text_parsed)).index(item))
        error_counts += 1

error 61
error 62
error 71
error 79
error 80
error 82
error 83
error 84
error 85
error 87
error 89
error 90
error 91
error 93
error 175
error 176
error 219
error 258
error 259
error 261
error 262
error 264
error 265
error 267
error 268
error 387
error 465
error 468
error 478
error 480
error 481
error 493
error 496
error 620
error 621
error 622
error 623
error 624
error 626
error 628
error 629
error 631
error 632
error 633
error 634
error 639
error 640
error 721
error 726
error 729
error 731
error 734
error 735
error 740
error 860
error 861
error 864
error 865
error 866
error 868
error 870
error 874
error 902
error 904
error 912
error 914
error 922
error 924
error 925
error 928
error 938
error 939
error 978
error 1018
error 1079
error 1091
error 1139
error 1214
error 1220
error 1283
error 1285
error 1291
error 1378
error 1379
error 1400
error 1420
error 1432
error 1500
error 1553
error 1591
error 1683
error 1691
error 1693
error 1697
error 1701
error 1702
error 1709
error 1710
error 171

In [35]:
error_counts

242

In [41]:
# Needs checking

wrong_sample_numbers = []

for index, sample in enumerate(found_sample_numbers):
    if index == 0:
        old_sample = sample
        continue
    else:
        if sample - old_sample == 1:
            old_sample = sample
            continue
        else:
            wrong_sample_numbers.append(index)
            old_sample = old_sample + 1

In [42]:
wrong_sample_numbers

[61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 

In [12]:
correct_lines(correct_lines(text_parsed))

['1- (44°31\'30",138 37\'30"). Granite alaskitic. K2 (V.N.Musin,1970). ',
 '2- (54°12\'10" ,119°24\'). Granite leucocratic. Tr. V.I.Zhigalova (E.A.Ivanov,1969). ',
 "3- (62°36',155°36'). Granite leucocratic coarse-grained. K2'. Left-Omsukchan massif. P.M.Bosek (O.S.Gracheva,1948). ",
 "4- (61°35',146°02'). · Granite leucocratic. K1. Buksandzhin massif. A.Kh.Brovtman (A.F.Mikhaylov,1948). ",
 "5- (68°55',164°24'). Granite-porphyry micropegmatitic. K1• Attykveem massif. L.G.Semenova. 0th. :S-0.16 (A. I.Sadovsky, 1963). ",
 "6- (50°42',113°42'). Bt granite porphyraceous medium-coarse-grained. J3. Sakhanay massif O.S.Beloborod'ko (N.N.Chaban,1966). ",
 "7- (47°30' ,136°27.'). Bt granite. K2. N.A.Krivitskaya (A.A.Golovneva,1950). ",
 "8- (48°14 ~ ,137°28'). Granite fine-grained. K2 (V.A.Yarmolyuk,1949). · ",
 "9- (50°16',112°59'). Granite leucocratic fine-grained. J3. Ongotson stock. M.I.Romanova (N.K.Dmitrochenko,1968). ",
 "10- (51°03',113°33'). Granite leucocratic fine-grained. J3. ,Kala

In [30]:
import re

regex = re.compile("[0-9]+-")

actual_lines = []

for index, test in enumerate(test_part):
    try: 
        regex.search(test).group()
        actual_lines.append(test)
    except:
        print("not a new line")
        actual_lines[-1] = test_part[index - 1] + test_part[index]

'1-'

'2-'

'3-'

not a new line


'4-'

not a new line


'5-'

not a new line


'6-'

not a new line


In [29]:
actual_lines

['1- (44°31\'30",138 37\'30"). Granite alaskitic. K2 (V.N.Musin,1970). ',
 '2- (54°12\'10" ,119°24\'). Granite leucocratic. Tr. V.I.Zhigalova (E.A.Ivanov,1969). ',
 "3- (62°36',155°36'). Granite leucocratic coarse-grained. K2'. Left-Omsukchan massif. P.M.Bosek (O.S.Gracheva,1948). ",
 "4- (61°35',146°02'). · Granite leucocratic. K1. Buksandzhin massif. A.Kh.Brovtman (A.F.Mikhaylov,1948). ",
 "5- (68°55',164°24'). Granite-porphyry micropegmatitic. K1• Attykveem massif. L.G.Semenova. 0th. :S-0.16 (A. I.Sadovsky, 1963). ",
 "6- (50°42',113°42'). Bt granite porphyraceous medium-coarse-grained. J3. Sakhanay massif O.S.Beloborod'ko (N.N.Chaban,1966). "]